In [1]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../src/')
import models
import generation_utils
import tokenizer
import data_utils

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
device = torch.device('cuda')

In [4]:
checkpoint_path = '../models/base_15/checkpoints/176_0.1316.pt'

In [5]:
checkpoint = torch.load(checkpoint_path)

In [6]:
args = checkpoint['args']

In [7]:
args['data']['data_loc']

'./data/2^16.json'

In [8]:
data_utils.gfm = data_utils.GlobalFactorMapping(data_path = '.' + args['data']['data_loc'] if args['data']['data_loc'].endswith('.json') else \
                                          args['data']['data_loc'] + '2^%d.json'%args['data']['max_pow'])

In [9]:
t = tokenizer.Tokenizer(base = args['data']['base'])

In [10]:
args.keys()

dict_keys(['data', 'model_args', 'optimizer', 'scheduler', 'loader', 'io', 'metrics', 'verbose', 'tokenizer'])

In [11]:
model = models.Factorizer(n_tokens = args['tokenizer']['n_tokens'], 
                          pad_token_id = args['tokenizer']['pad_token_id'],
                          **args['model_args'])
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

Factorizer(
  (embedding): TransformerEmbedding(
    (embedding): Embedding(19, 128)
    (pe): PositionalEncoding(
      (dropout): Dropout(p=0.05, inplace=False)
    )
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
          )
          (linear1): Linear(in_features=128, out_features=512, bias=True)
          (dropout): Dropout(p=0.05, inplace=False)
          (linear2): Linear(in_features=512, out_features=128, bias=True)
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.05, inplace=False)
          (dropout2): Dropout(p=0.05, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            

In [15]:
import importlib
importlib.reload(generation_utils)

<module 'generation_utils' from '../src\\generation_utils.py'>

In [16]:
example_row = generation_utils.factor(17, args['data']['base'], model, t, device, args['model_args']['max_decode_size'], n_beams = 10)

In [17]:
example_row

,target_num,target_is_prime,input_string,target_str,target_factor_list,pred_str,beam_idx,log_prob,n_target_factors,pred_same_as_target,pred_factor_list,n_pred_factors,product,correct_product,correct_factorization,num_prime_factors_pred,percent_prime_factors_pred
0,17,True,12,>12.,[17],>12._____,0,-0.275626,1,True,[17],1,17,True,True,1,1.0
1,17,True,12,>12.,[17],>7x27.___,1,-2.806215,1,False,"[7, 37]",2,259,False,False,2,1.0
2,17,True,12,>12.,[17],>Bx18.___,2,-2.892918,1,False,"[11, 23]",2,253,False,False,2,1.0
3,17,True,12,>12.,[17],>Dx14.___,3,-3.085363,1,False,"[13, 19]",2,247,False,False,2,1.0
4,17,True,12,>12.,[17],>27._____,4,-3.773743,1,False,[37],1,37,False,False,1,1.0
5,17,True,12,>12.,[17],>2x2x2x2.,5,-3.857347,1,False,"[2, 2, 2, 2]",4,16,False,False,4,1.0
6,17,True,12,>12.,[17],>Bx3.____,6,-4.615718,1,False,"[11, 3]",2,33,False,False,2,1.0
7,17,True,12,>12.,[17],>32._____,7,-4.645571,1,False,[47],1,47,False,False,1,1.0
8,17,True,12,>12.,[17],>38._____,8,-5.638335,1,False,[53],1,53,False,False,1,1.0
9,17,True,12,>12.,[17],>21._____,9,-6.181497,1,False,[31],1,31,False,False,1,1.0
